In [ ]:
import main
import random
import argparse
from cgi import test
from itertools import count
from pathlib import Path
import random
import numpy as np
from PIL import Image

import PIL.ImageOps
from numpy import quantile  

from get_mnist import load_mnist

In [ ]:
import argparse
from cgi import test
from itertools import count
from pathlib import Path
import random

from PIL import Image

import PIL.ImageOps
from numpy import quantile  
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from get_mnist import load_mnist
from num_seq_generator import generate_numbers_sequence

In [ ]:
# ........................Gendata..........................
for i in range(1000):
    x= random.randint(100000,999999)
    out_path= Path("D:/ANlab/sequence-generator/data")
    sequence= x
    sequence = list(map(int, str(sequence)))
    train_imgs, train_labels, _, _ = load_mnist(out_path)
    
    number_sequence_image = generate_numbers_sequence(
        sequence,
        (0,40),
        150,
        train_imgs,
        train_labels,
    )
    im = Image.fromarray(number_sequence_image * 255)
    im = im.convert("L")
    im = PIL.ImageOps.invert(im)
    # print(args.sequence)
    count= random.randint(100000000000000,999999999999999)
    name = 'datagen_train_1000'+'/' + str(x)+"_" + str(count) +'.png'
    im.save(name, quality= 100)

In [ ]:
import os
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dropout, Dense, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Model
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import Sequential

In [ ]:
data_folder = "D:/ANlab/sequence-generator/datagen_train_1000"
train_image_paths= []
image_label_train=[]
for path in os.listdir(data_folder):
    train_image_paths.append(data_folder + "/" + path)
    img_name = path[0:len(path)-20]
    image_label_train.append(img_name)  

len(image_label_train)

In [ ]:
CHAR_VECTOR= "0123456789"
letters = [letter for letter in CHAR_VECTOR]

num_classes = len(letters) + 1
max_text_len= 6
img_w, img_h  = 128, 64

print(num_classes)

In [ ]:
char_list = sorted(CHAR_VECTOR)
def encode_to_labels(txt):

    dig_lst = [] 
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            break
            # print(char)
    
    return pad_sequences([dig_lst], maxlen=max_text_len, padding='post', value = 11)[0]

In [ ]:
padded_image_texts_train = list(map(encode_to_labels, image_label_train))

print(padded_image_texts_train[100])
image_label_train[100]

In [ ]:
def process_single_sample(img_path, label):
    
    # 1. Read image
    img = tf.io.read_file(img_path)

    # 2. Decode and convert to grayscale
    img = tf.io.decode_jpeg(img, channels=1)

    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)

    # 4. Resize to the desired size
    img = tf.image.resize(img, [64,128])
    img = tf.transpose(img, perm=(1,0,2))
    label = tf.strings.as_string(label)
    label = tf.strings.to_number(label, tf.int32)
    return { "Input": img, "Label": label }
    # return img, label

In [ ]:

trainAug = Sequential([
	# preprocessing.Rescaling(scale=1.0 / 255),

	# preprocessing.RandomZoom(
		# height_factor=(-0.05, -0.15),
		# width_factor=(-0.05, -0.15)),
	# preprocessing.RandomRotation(0.01)
])

In [ ]:
batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((train_image_paths, padded_image_texts_train))

train_dataset = (
    train_dataset.map(
        process_single_sample, num_parallel_calls=tf.data.experimental.AUTOTUNE
    )
    # .shuffle(buffer_size=1000)
    # .map(lambda x, y: (trainAug(x), y),
	# 	 num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
)

In [ ]:
train_dataset

In [ ]:
# from tensorflow.keras import layers
# from tensorflow.keras import models
# import os
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.applications.resnet50 import preprocess_input
# from tensorflow.keras.preprocessing import image



# def batch_activate(x):
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation('relu')(x)
#     return x
# def convolution_block(x,
#                       filters,
#                       size,
#                       strides=(1, 1),
#                       padding='same',
#                       activation=True):
#     x = layers.Conv2D(filters, size, strides=strides, padding=padding)(x)
#     if activation:
#         x = batch_activate(x)
#     return x

# def residual_block(block_input,
#                    num_filters=16,
#                    use_batch_activate=False):
#     x = batch_activate(block_input)
#     x = convolution_block(x, num_filters, (3, 3))
#     x = convolution_block(x, num_filters, (3, 3), activation=False)
#     x = layers.Add()([x, block_input])
#     if use_batch_activate:
#         x = batch_activate(x)
#     return x
# def resnet_backbone(start_neurons=32,
#                     dropout_rate=0.1):
#     input_layer = layers.Input(
#         name='input_image',
#         shape=(128,32,1),
#         dtype='float32'
#     )

#     for index, i in enumerate([1, 2, 2, 4, 8]):
#         if index == 0:
#             inner = input_layer
#         inner = layers.Conv2D(start_neurons * i, (3,3),
#                               activation=None, padding="same")(inner)
#         inner = residual_block(inner, start_neurons * i)
#         inner = residual_block(inner, start_neurons * i, True)

#         if i <=2:
#             inner = layers.MaxPooling2D((2,2))(inner)

#         if dropout_rate:
#             inner = layers.Dropout(dropout_rate)(inner)
    
#     inner = Reshape(target_shape=((32,512)), name='reshape')(inner)  # (None, 32, 2048)
#     inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)  # (None, 32, 64)
#     inner1 = Dropout(0.1)(inner)

#     x = layers.Bidirectional(
#         layers.LSTM(units=256, return_sequences=True, dropout=0.1), name='bi_lstm1')(inner1)
#     x = layers.Bidirectional(
#         layers.LSTM(units=256, return_sequences=True, dropout=0.1), name='bi_lstm2')(x)
#     x = layers.Dense(units=num_classes, name='logits')(x)

    
#     return Model(inputs=input_layer, outputs=x)

# model = resnet_backbone()
# model.summary()

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.layers import Reshape, Lambda, BatchNormalization
from tensorflow.keras.layers import Add, Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
K.set_learning_phase(0)

# Build an endpoint layer for implementing CTC loss.
class CTCLayer( layers.Layer ):
    def __init__( self, name=None, **kwargs ):
        super().__init__( name=name )
        self.loss_fn = K.ctc_batch_cost

    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it to the layer using `self.add_loss()`.
        batch_len = tf.cast( tf.shape(y_true)[0], dtype='int64' )
        input_length = tf.cast( tf.shape(y_pred)[1], dtype='int64' )
        label_length = tf.cast( tf.shape(y_true)[1], dtype='int64' )
        
        input_length = input_length*tf.ones( shape=(batch_len,1), dtype='int64' )
        label_length = label_length*tf.ones( shape=(batch_len,1), dtype='int64' )

        loss = self.loss_fn( y_true, y_pred, input_length, label_length )
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred

def get_Model(training):
    input_shape = (128,64, 1)     # (128, 64, 1)
    labels = Input( shape=(None,), dtype='float32', name="Label" )
    # labels = layers.Input(name="label", shape=(None,), dtype="float32")
    # Make Networkw
    inputs = Input(name='Input', shape=input_shape, dtype='float32')  # (None, 128, 64, 1)

    # Convolution layer (VGG)
    inner = Conv2D(64, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(inputs)  # (None, 128, 64, 64)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)  # (None,64, 32, 64)

    inner = Conv2D(128, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(inner)  # (None, 64, 32, 128)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)  # (None, 32, 16, 128)

    inner = Conv2D(256, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(inner)  # (None, 32, 16, 256)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = Conv2D(256, (3, 3), padding='same', name='conv4', kernel_initializer='he_normal')(inner)  # (None, 32, 16, 256)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(1, 2), name='max3')(inner)  # (None, 32, 8, 256)


    inner = Conv2D(512, (3, 3), padding='same', name='conv5', kernel_initializer='he_normal')(inner)  # (None, 32, 8, 512)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = Conv2D(512, (3, 3), padding='same', name='conv6')(inner)  # (None, 32, 8, 512)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)
    inner = MaxPooling2D(pool_size=(1, 2), name='max4')(inner)  # (None, 32, 4, 512)

    inner = Conv2D(512, (2, 2), padding='same', kernel_initializer='he_normal', name='con7')(inner)  # (None, 32, 4, 512)
    inner = BatchNormalization()(inner)
    inner = Activation('relu')(inner)



    # CNN to RNN
    inner = Reshape(target_shape=((32,2048)), name='reshape')(inner)  # (None, 32, 2048)
    inner = Dense(128, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)  # (None, 32, 64)
    inner1 = Dropout(0.1)(inner)


    x = layers.Bidirectional(
        layers.LSTM(units=256, return_sequences=True), name='bi_lstm1')(inner)
    x = layers.Bidirectional(
        layers.LSTM(units=256, return_sequences=True), name='bi_lstm2')(x)
    # x = layers.Bidirectional(
    #     layers.LSTM(units=128, return_sequences=True), name='bi_lstm3')(x)
    x = Dense( num_classes, activation='softmax', name='Softmax' )(x) 

    output = CTCLayer( name='CTC_Loss' )(labels, x)

    if training:
      return Model( inputs=[inputs,labels], outputs=[output], name='CRNN_Model_with_CTC_LOSS' ) 

    
model = get_Model(training=True)
model.summary()



In [ ]:
optimizer = tf.keras.optimizers.Adam()
model.compile( optimizer=optimizer)

In [ ]:
# class CTCLoss(keras.losses.Loss):
#     """ A class that wraps the function of tf.nn.ctc_loss. 
    
#     Attributes:
#         logits_time_major: If False (default) , shape is [batch, time, logits], 
#             If True, logits is shaped [time, batch, logits]. 
#         blank_index: Set the class index to use for the blank label. default is
#             -1 (num_classes - 1). 
#     """

#     def __init__(self, logits_time_major=False, name='ctc_loss'):
#         super().__init__(name=name)
#         self.logits_time_major = logits_time_major
        
#     def call(self, y_true, y_pred):
#         """ 
#             Computes CTC (Connectionist Temporal Classification) loss. 
#         """
#         y_true = tf.cast(y_true, tf.int32)
#         logit_length = tf.fill([tf.shape(y_pred)[0]], tf.shape(y_pred)[1])
#         label_length = tf.fill([tf.shape(y_true)[0]], tf.shape(y_true)[1])
#         loss = tf.nn.ctc_loss(
#             labels=y_true,
#             logits=y_pred,
#             label_length=label_length,
#             logit_length=logit_length,
#             logits_time_major=self.logits_time_major,
            
#         )
#         return tf.math.reduce_mean(loss)

In [ ]:
# strategy = tf.distribute.get_strategy()
# with strategy.scope():
#     # optimizer
#     optimizer = tf.keras.optimizers.Adam()
#     # model=build_model()
#     # compile
#     model.compile(optimizer=optimizer,loss=CTCLoss())

In [ ]:
# checkpoint_path = "D:/ANlab/sequence-generator/CP/best.hdf5"
# checkpoint_dir = os.path.dirname(checkpoint_path)
# checkpoint_dir
# checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss', verbose=1, mode='min', save_best_only=True, save_weights_only=False)

# callbacks_list = [checkpoint, EarlyStopping(monitor='val_loss', min_delta=0.001, patience=30, mode='min', verbose=1)]



In [ ]:
history = model.fit(train_dataset,
                    epochs = 5,
                    # validation_data=validation_dataset,
                    verbose = 1,
                    # callbacks = callbacks_list,
                    initial_epoch=0)

In [ ]:
model.save_weights('D:/ANlab/sequence-generator/CP/best.hdf5')

In [ ]:
model.load_weights('D:/ANlab/sequence-generator/CP/best.hdf5')

In [ ]:
prediction_model = keras.models.Model(
    model.get_layer(name="Input").input, model.get_layer(name="Softmax").output
)
prediction_model.summary()

In [ ]:
from tensorflow.keras import backend as K

Characters={'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}
char_to_num = layers.experimental.preprocessing.StringLookup(
                  vocabulary=sorted(list(Characters)), num_oov_indices=0, mask_token=None )
num_to_char = layers.experimental.preprocessing.StringLookup(
                  vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True )

def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = K.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][:,:6]
    # Iterate over the results and get back the text
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode('utf-8')
        # res= res.numpy()
        output_text.append(res)
    return output_text

In [ ]:
correct = 0
error= 0
for batch in train_dataset.take(1):
  images = batch['Input']
  labels = batch['Label']
  # print(images[0], labels[0])
  preds = prediction_model.predict(images)
  preds= decode_batch_predictions(preds)
  orig_texts = []
  for label in labels:
        label = tf.strings.reduce_join(num_to_char(label)).numpy().decode('utf-8')
        orig_texts.append(label)
  # print(preds)
  # print(orig_texts)
  for i in range(len(orig_texts)):
    # print(preds[i])
    # print(orig_texts[i])
    if preds[i] == orig_texts[i]:
      correct+=1
    else:
      error += 1
print(error)
print(correct)
  